# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.  

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.23.0

Let's define a prompt outlining a dialogue chat bot.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Sure! What genre are you interested in? Fiction, non-fiction, fantasy, mystery, or something else?



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");


User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: I recommend "The Histories" by Herodotus. It's considered one of the foundational works of history and provides a fascinating account of ancient Greece and its interactions with other cultures.



In [9]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");


User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Histories," you'll learn about a variety of topics, including:

1. **The Greco-Persian Wars**: Herodotus provides detailed accounts of the conflicts between the Greek city-states and the Persian Empire.
2. **Cultural Practices**: He explores the customs, traditions, and lifestyles of different peoples, including the Egyptians, Scythians, and Persians.
3. **Geography**: The book includes descriptions of various regions and cities, offering insights into the ancient world’s geography.
4. **Political Systems**: You'll learn about the different forms of government and political structures in various city-states.
5. **Mythology and Religion**: Herodotus discusses the religious beliefs and mythological stories that shaped the cultures of the time.

Overall, it's a rich tapestry of historical events, cultural insights, and personal anecdotes that paint a vivid picture of ancient Greece and its world.



In [10]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: Many people find the **Greco-Persian Wars** particularly interesting. This topic not only involves dramatic battles and strategic maneuvers but also highlights themes of heroism, democracy, and the clash of civilizations. The stories of famous battles like Marathon and Thermopylae, along with the larger implications for Greek identity and unity, tend to captivate readers and spark discussions about history's impact on modern society.



In [20]:
await Chat("what kind of book did I ask for?");

Error: Microsoft.SemanticKernel.HttpOperationException: HTTP 429 (429)

Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-08-01-preview have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 7 seconds. Please contact Azure support service if you would like to further increase the default rate limit.
 ---> System.ClientModel.ClientResultException: HTTP 429 (429)

Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-08-01-preview have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 7 seconds. Please contact Azure support service if you would like to further increase the default rate limit.
   at Azure.AI.OpenAI.ClientPipelineExtensions.ProcessMessageAsync(ClientPipeline pipeline, PipelineMessage message, RequestOptions options)
   at Azure.AI.OpenAI.Chat.AzureChatClient.CompleteChatAsync(BinaryContent content, RequestOptions options)
   at Ope

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [15]:
Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Sure! What genre are you interested in? Fiction, non-fiction, fantasy, mystery, or something else?

User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: I recommend "The Histories" by Herodotus. It's considered one of the foundational works of history and provides a fascinating account of ancient Greece and its interactions with other cultures.

User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Histories," you'll learn about a variety of topics, including:

1. **The Greco-Persian Wars**: Herodotus provides detailed accounts of the conflicts between the Greek city-states and the Persian Empire.
2. **Cultural Practices**: He explores the customs, traditions, and lifestyles of different peoples, including the Egyptians, Scythians, and Persians.
3. **Geography**: The book includes descriptions of various regions and cities, offering insights into the anc